# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-04-22 08:29:05] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37034, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=1019844765, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=

[2025-04-22 08:29:16 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-22 08:29:16 TP0] Init torch distributed begin.
[2025-04-22 08:29:16 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-22 08:29:16 TP0] Load weight begin. avail mem=53.74 GB


[2025-04-22 08:29:16 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-22 08:29:17 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.41s/it]


...

[2025-04-22 08:29:20 TP0] 

CUDA Graph is DISABLED.
This will cause significant performance degradation.
CUDA Graph should almost never be disabled in most usage scenarios.
If you encounter OOM issues, please try setting --mem-fraction-static to a lower value (such as 0.8 or 0.7) instead of disabling CUDA Graph.



[2025-04-22 08:29:21 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-22 08:29:21] INFO:     Started server process [65506]
[2025-04-22 08:29:21] INFO:     Waiting for application startup.
[2025-04-22 08:29:21] INFO:     Application startup complete.
[2025-04-22 08:29:21] INFO:     Uvicorn running on http://0.0.0.0:37034 (Press CTRL+C to quit)
[2025-04-22 08:29:21] INFO:     127.0.0.1:52840 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-22 08:29:22] INFO:     127.0.0.1:52856 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-22 08:29:22 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:29:25] INFO:     127.0.0.1:52868 - "POST /generate HTTP/1.1" 200 OK
[2025-04-22 08:29:25] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-22 08:29:27 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:29:28 TP0] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 5.64, #queue-req: 0, 


[2025-04-22 08:29:28 TP0] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 108.67, #queue-req: 0, 


[2025-04-22 08:29:28 TP0] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 112.77, #queue-req: 0, 


[2025-04-22 08:29:29 TP0] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 108.38, #queue-req: 0, 


...

[2025-04-22 08:29:45 TP0] Decode batch. #running-req: 1, #token: 1892, token usage: 0.09, gen throughput (token/s): 106.81, #queue-req: 0, 


[2025-04-22 08:29:45 TP0] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 109.26, #queue-req: 0, 


[2025-04-22 08:29:46 TP0] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 106.81, #queue-req: 0, 


[2025-04-22 08:29:46 TP0] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 104.99, #queue-req: 0, 


[2025-04-22 08:29:46 TP0] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 110.17, #queue-req: 0, 
[2025-04-22 08:29:47] INFO:     127.0.0.1:52878 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-22 08:29:47 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:29:47 TP0] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 77.13, #queue-req: 0, 


[2025-04-22 08:29:47 TP0] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 108.21, #queue-req: 0, 


[2025-04-22 08:29:48 TP0] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 105.41, #queue-req: 0, 


[2025-04-22 08:29:48 TP0] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 106.98, #queue-req: 0, 


...

[2025-04-22 08:30:08 TP0] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 103.93, #queue-req: 0, 


[2025-04-22 08:30:09 TP0] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 104.33, #queue-req: 0, 


[2025-04-22 08:30:09 TP0] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 87.61, #queue-req: 0, 


[2025-04-22 08:30:09 TP0] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 107.48, #queue-req: 0, 


[2025-04-22 08:30:10] INFO:     127.0.0.1:52878 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-22 08:30:10 TP0] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:30:10 TP0] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 62.98, #queue-req: 0, 


[2025-04-22 08:30:11 TP0] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 65.57, #queue-req: 0, 


[2025-04-22 08:30:11 TP0] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 66.49, #queue-req: 0, 


[2025-04-22 08:30:12] INFO:     127.0.0.1:52878 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-22 08:30:12 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:30:12 TP0] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 76.39, #queue-req: 0, 


[2025-04-22 08:30:12 TP0] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 108.49, #queue-req: 0, 


[2025-04-22 08:30:13 TP0] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 106.62, #queue-req: 0, 


[2025-04-22 08:30:13 TP0] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 109.68, #queue-req: 0, 


...

[2025-04-22 08:30:14 TP0] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 107.95, #queue-req: 0, 


[2025-04-22 08:30:14 TP0] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 111.13, #queue-req: 0, 


[2025-04-22 08:30:14 TP0] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 107.22, #queue-req: 0, 


[2025-04-22 08:30:15 TP0] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 98.19, #queue-req: 0, 


[2025-04-22 08:30:15] INFO:     127.0.0.1:52878 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-22 08:30:15 TP0] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:30:16 TP0] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 46.69, #queue-req: 0, 


[2025-04-22 08:30:16 TP0] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 92.65, #queue-req: 0, 


[2025-04-22 08:30:17 TP0] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 76.75, #queue-req: 0, 


[2025-04-22 08:30:17 TP0] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 99.83, #queue-req: 0, 


...

[2025-04-22 08:30:18 TP0] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 92.03, #queue-req: 0, 


[2025-04-22 08:30:18 TP0] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 96.89, #queue-req: 0, 


[2025-04-22 08:30:19 TP0] Decode batch. #running-req: 1, #token: 775, token usage: 0.04, gen throughput (token/s): 104.96, #queue-req: 0, 


[2025-04-22 08:30:19 TP0] Decode batch. #running-req: 1, #token: 815, token usage: 0.04, gen throughput (token/s): 105.88, #queue-req: 0, 


[2025-04-22 08:30:19 TP0] Decode batch. #running-req: 1, #token: 855, token usage: 0.04, gen throughput (token/s): 104.23, #queue-req: 0, 
[2025-04-22 08:30:20] INFO:     127.0.0.1:52878 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-04-22 08:30:20 TP0] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:30:20 TP0] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, gen throughput (token/s): 46.66, #queue-req: 0, 


[2025-04-22 08:30:21 TP0] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, gen throughput (token/s): 108.58, #queue-req: 0, 


[2025-04-22 08:30:21 TP0] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, gen throughput (token/s): 108.55, #queue-req: 0, 


[2025-04-22 08:30:21 TP0] Decode batch. #running-req: 1, #token: 166, token usage: 0.01, gen throughput (token/s): 111.41, #queue-req: 0, 


...

[2025-04-22 08:30:39 TP0] Decode batch. #running-req: 1, #token: 1926, token usage: 0.09, gen throughput (token/s): 103.54, #queue-req: 0, 


[2025-04-22 08:30:39 TP0] Decode batch. #running-req: 1, #token: 1966, token usage: 0.10, gen throughput (token/s): 101.64, #queue-req: 0, 


[2025-04-22 08:30:40 TP0] Decode batch. #running-req: 1, #token: 2006, token usage: 0.10, gen throughput (token/s): 99.13, #queue-req: 0, 


[2025-04-22 08:30:40 TP0] Decode batch. #running-req: 1, #token: 2046, token usage: 0.10, gen throughput (token/s): 98.20, #queue-req: 0, 


[2025-04-22 08:30:40] INFO:     127.0.0.1:42674 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Paris is a big city, but I think it\'s not as large as Tokyo or London. Maybe around 10 square kilometers? I\'m not sure, but that seems 

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-04-22 08:30:40 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 08:30:40 TP0] Decode batch. #running-req: 1, #token: 23, token usage: 0.00, gen throughput (token/s): 103.58, #queue-req: 0, 


[2025-04-22 08:30:41 TP0] Decode batch. #running-req: 1, #token: 63, token usage: 0.00, gen throughput (token/s): 108.77, #queue-req: 0, 


[2025-04-22 08:30:41 TP0] Decode batch. #running-req: 1, #token: 103, token usage: 0.01, gen throughput (token/s): 106.26, #queue-req: 0, 


[2025-04-22 08:30:42 TP0] Decode batch. #running-req: 1, #token: 143, token usage: 0.01, gen throughput (token/s): 110.22, #queue-req: 0, 


[2025-04-22 08:30:42 TP0] Decode batch. #running-req: 1, #token: 183, token usage: 0.01, gen throughput (token/s): 105.23, #queue-req: 0, 


...

[2025-04-22 08:30:46 TP0] Decode batch. #running-req: 1, #token: 623, token usage: 0.03, gen throughput (token/s): 106.57, #queue-req: 0, 


[2025-04-22 08:30:46 TP0] Decode batch. #running-req: 1, #token: 663, token usage: 0.03, gen throughput (token/s): 107.35, #queue-req: 0, 


[2025-04-22 08:30:47 TP0] Decode batch. #running-req: 1, #token: 703, token usage: 0.03, gen throughput (token/s): 108.35, #queue-req: 0, 


[2025-04-22 08:30:47 TP0] Decode batch. #running-req: 1, #token: 743, token usage: 0.04, gen throughput (token/s): 97.98, #queue-req: 0, 


[2025-04-22 08:30:48 TP0] Decode batch. #running-req: 1, #token: 783, token usage: 0.04, gen throughput (token/s): 78.34, #queue-req: 0, 
[2025-04-22 08:30:48] INFO:     127.0.0.1:42014 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-04-22 08:30:48 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 08:30:48 TP0] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:30:48 TP0] Decode batch. #running-req: 3, #token: 92, token usage: 0.00, gen throughput (token/s): 132.54, #queue-req: 0, 


[2025-04-22 08:30:49 TP0] Decode batch. #running-req: 3, #token: 212, token usage: 0.01, gen throughput (token/s): 227.89, #queue-req: 0, 


[2025-04-22 08:30:49 TP0] Decode batch. #running-req: 3, #token: 332, token usage: 0.02, gen throughput (token/s): 233.18, #queue-req: 0, 


[2025-04-22 08:30:50 TP0] Decode batch. #running-req: 3, #token: 452, token usage: 0.02, gen throughput (token/s): 299.97, #queue-req: 0, 


[2025-04-22 08:30:50 TP0] Decode batch. #running-req: 3, #token: 572, token usage: 0.03, gen throughput (token/s): 307.94, #queue-req: 0, 
[2025-04-22 08:30:50] INFO:     127.0.0.1:51780 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any othe

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-04-22 08:30:50 TP0] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 08:30:51 TP0] Decode batch. #running-req: 1, #token: 32, token usage: 0.00, gen throughput (token/s): 170.34, #queue-req: 0, 


[2025-04-22 08:30:51 TP0] Decode batch. #running-req: 1, #token: 72, token usage: 0.00, gen throughput (token/s): 108.85, #queue-req: 0, 


[2025-04-22 08:30:51 TP0] Decode batch. #running-req: 1, #token: 112, token usage: 0.01, gen throughput (token/s): 108.75, #queue-req: 0, 


[2025-04-22 08:30:52 TP0] Decode batch. #running-req: 1, #token: 152, token usage: 0.01, gen throughput (token/s): 110.43, #queue-req: 0, 


...

[2025-04-22 08:31:08 TP0] Decode batch. #running-req: 1, #token: 1912, token usage: 0.09, gen throughput (token/s): 106.24, #queue-req: 0, 


[2025-04-22 08:31:09 TP0] Decode batch. #running-req: 1, #token: 1952, token usage: 0.10, gen throughput (token/s): 106.56, #queue-req: 0, 


[2025-04-22 08:31:09 TP0] Decode batch. #running-req: 1, #token: 1992, token usage: 0.10, gen throughput (token/s): 106.73, #queue-req: 0, 


[2025-04-22 08:31:09 TP0] Decode batch. #running-req: 1, #token: 2032, token usage: 0.10, gen throughput (token/s): 108.31, #queue-req: 0, 


[2025-04-22 08:31:10] INFO:     127.0.0.1:51792 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-04-22 08:31:10 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 08:31:10 TP0] Decode batch. #running-req: 1, #token: 38, token usage: 0.00, gen throughput (token/s): 102.66, #queue-req: 0, 


[2025-04-22 08:31:10 TP0] Decode batch. #running-req: 1, #token: 78, token usage: 0.00, gen throughput (token/s): 108.80, #queue-req: 0, 


[2025-04-22 08:31:11 TP0] Decode batch. #running-req: 1, #token: 118, token usage: 0.01, gen throughput (token/s): 86.80, #queue-req: 0, 


[2025-04-22 08:31:11 TP0] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, gen throughput (token/s): 105.42, #queue-req: 0, 


[2025-04-22 08:31:11 TP0] Decode batch. #running-req: 1, #token: 198, token usage: 0.01, gen throughput (token/s): 106.96, #queue-req: 0, 


...

[2025-04-22 08:31:18 TP0] Decode batch. #running-req: 1, #token: 878, token usage: 0.04, gen throughput (token/s): 107.40, #queue-req: 0, 


[2025-04-22 08:31:18 TP0] Decode batch. #running-req: 1, #token: 918, token usage: 0.04, gen throughput (token/s): 105.02, #queue-req: 0, 


[2025-04-22 08:31:19 TP0] Decode batch. #running-req: 1, #token: 958, token usage: 0.05, gen throughput (token/s): 104.43, #queue-req: 0, 


[2025-04-22 08:31:19 TP0] Decode batch. #running-req: 1, #token: 998, token usage: 0.05, gen throughput (token/s): 104.61, #queue-req: 0, 


[2025-04-22 08:31:19] INFO:     127.0.0.1:44746 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-04-22 08:31:19] Child process unexpectedly failed with an exit code 9. pid=66146


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.40s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.33s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.34s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 2005-05-24

The capital of France is Paris. It was the capital from 1870 to 1960. In 1960, the capital was moved to Paris.

Wait, that seems repetitive. Maybe I should check the source.

Yes, according to the French government's official website, Paris was the capital from 1870 until it was moved to another city. Wait, no, that doesn't make sense. I think in 1960, they moved the capital.

I think I got confused. Let me try again. The capital
Prompt: Give me the information of the capital of Germany.
Generated text: 
Alright, so I need to figure out the capital of Germany. Hmm, I remember that Germany is a country in Europe, right? I think it's one of the bigger countries in Europe, maybe comparable to France or Italy. But I'm not entirely sure about its capital.

Wait, I think I've heard that the capital of Germany is Berlin. Yeah, that sounds familiar. I've seen a lot of the city around Berlin, like the Brandenb

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its population, cultural significance, major attractions, and transport systems.

8.1.1: What is the population of London?

8.1.2: What is London known for in terms of culture?

8.1.3: What are some major attractions in London?

8.1.4: What transport systems are available in London?

Please provide detailed information for each question.

8.1.1: Population of London

London is the capital city of England and the United Kingdom, with a population exceeding 9 million. The exact figure is approximately 9.6 million, making it the most populous capital city globally.


Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economy, culture, and history.

 Paris, capital of France, is a major global city known for its rich history, vibrant culture, and iconic landmarks. It is located in western France, bordered by the Seine River to the east, and is surround

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Okay, I need to provide information about the capital of France in JSON format. Let me start by recalling the basic details. The capital is Paris, so that's the first piece of information. Next, I should include some key facts or notable features. Paris is known as the "City of Light," famous for its landmarks like the Eiffel Tower and the Louvre Museum. I should mention its rich history, being the birthplace of famous figures like Victor Hugo and Ernest Hemingway, and its cultural significance as a Parisian art district, Madeleine Quarter. 

I also remember that Paris is the political center of France and has a population exceeding 2 million. Including its geographical location, it's located in northern France, near the Seine River and the Eiffel Tower. Additionally, Paris is a major transportation hub with its extensive metro system and is pa

In [19]:
llm.shutdown()